In [1]:
import os

# Đường dẫn thư mục input
input_dir = '/kaggle/input/encoding-image-and-testing'

# Biến đếm ảnh và tìm file pkl
image_count = 0
feature_vector_path = None

# Duyệt qua tất cả các file
for root, dirs, files in os.walk(input_dir):
    for file in files:
        # Đếm ảnh dựa vào phần mở rộng
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_count += 1
        # Tìm file feature_vectors.pkl
        if file == 'feature_vectors.pkl':
            feature_vector_path = os.path.join(root, file)

# Kết quả
print(f"Tổng số ảnh: {image_count}")
if feature_vector_path:
    print(f"Đã tìm thấy file feature_vectors.pkl tại: {feature_vector_path}")
else:
    print("Không tìm thấy file feature_vectors.pkl")


Tổng số ảnh: 31395
Đã tìm thấy file feature_vectors.pkl tại: /kaggle/input/encoding-image-and-testing/feature_vectors.pkl


In [2]:
import os
import glob
import random
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# ==== Dataset Class ====
class TripletDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.image_paths = glob.glob(os.path.join(data_dir, "*.jpg"))
        self.class_to_images = {}

        for path in self.image_paths:
            cls = os.path.basename(path).split("_")[0]
            self.class_to_images.setdefault(cls, []).append(path)

        self.classes = list(self.class_to_images.keys())

    def __len__(self):
        return 10000

    def __getitem__(self, idx):
        anchor_class = random.choice(self.classes)
        positive_class = anchor_class
        negative_class = random.choice([cls for cls in self.classes if cls != anchor_class])

        anchor, positive = random.sample(self.class_to_images[anchor_class], 2)
        negative = random.choice(self.class_to_images[negative_class])

        anchor_img = Image.open(anchor).convert("RGB")
        positive_img = Image.open(positive).convert("RGB")
        negative_img = Image.open(negative).convert("RGB")

        if self.transform:
            anchor_img = self.transform(anchor_img)
            positive_img = self.transform(positive_img)
            negative_img = self.transform(negative_img)

        return anchor_img, positive_img, negative_img

# ==== Embedding Network ====
class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        base = models.resnet50(pretrained=True)
        self.backbone = nn.Sequential(*list(base.children())[:-1])
        self.fc = nn.Linear(2048, 256)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.normalize(x, p=2, dim=1)

# ==== Triplet Loss ====
def triplet_loss(anchor, positive, negative, margin=1.0):
    pos_dist = F.pairwise_distance(anchor, positive)
    neg_dist = F.pairwise_distance(anchor, negative)
    return torch.relu(pos_dist - neg_dist + margin).mean()

# ==== Train Function ====
def train_embedding_model(data_dir, batch_size=32, epochs=5, lr=1e-4, output_path="resnet50_triplet.pth"):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    dataset = TripletDataset(data_dir, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = EmbeddingNet().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for anchor, pos, neg in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            anchor, pos, neg = anchor.to(device), pos.to(device), neg.to(device)
            out_a = model(anchor)
            out_p = model(pos)
            out_n = model(neg)

            loss = triplet_loss(out_a, out_p, out_n)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    torch.save(model.state_dict(), output_path)
    print(f"✅ Model saved to: {output_path}")
    return model


In [3]:
train_embedding_model("/kaggle/input/encoding-image-and-testing/cropped_images",batch_size=54, epochs=70, lr=1e-4,)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 220MB/s]
Epoch 1/70: 100%|██████████| 186/186 [05:41<00:00,  1.84s/it]


Epoch 1, Loss: 0.4738


Epoch 2/70: 100%|██████████| 186/186 [05:12<00:00,  1.68s/it]


Epoch 2, Loss: 0.3978


Epoch 3/70: 100%|██████████| 186/186 [04:58<00:00,  1.61s/it]


...

Epoch 69/70: 100%|██████████| 186/186 [04:42<00:00,  1.52s/it]


Epoch 69, Loss: 0.0692


Epoch 70/70: 100%|██████████| 186/186 [04:45<00:00,  1.54s/it]

Epoch 70, Loss: 0.0693
✅ Model saved to: resnet50_triplet.pth


EmbeddingNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2